<a href="https://colab.research.google.com/github/joshtburdick/misc/blob/master/plog/Factoring3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Further attempt at factoring

This is to test a simpler variant of the factoring method using loopy belief propagation. But without the loopy belief propagation.

The main idea is, given $n$, to solve $n=(a+b)(a-b) \mod m$, where $m = \prod p_i$ for $P$ (smallish) primes. Then check $\mathrm{GCD}(n, x)$ for $2^{2P}$ numbers $x$ which are derived from $a$ and $b$ using the Chinese Remainder Theorem.

N.B.: This seems fairly impractical, as it requires computing GCD $2^{2P}$ times, for multiple choices of $a+b$ and $a-b$.

In [1]:
!pip install --quiet modulo

In [2]:
import itertools
import math

import numpy as np

from modulo import modulo

We'll need some Chinese Remainder Theorem utilities.

In [3]:
def solve_mod_primes(x_mod, primes):
    """Given what x is (mod some primes), solve for x.

    x_mod: an array of small integers, such that x % primes[i] == x_mod[i]
    primes: an array of primes

    Returns: x, in the range 1 <= x <= product(primes),
        satisfying x % primes[i] == xmod[i].
    """
    x = modulo(x_mod[0], primes[0])
    for i in range(1, len(primes)):
        x &= modulo(x_mod[i], primes[i])
    return int(x)

Let $n$ be the number to be factored (WLOG assume $n$ is odd). We want to write $n=(a+b)(a-b)$.

**Somewhat dubious** conjecture: WLOG we can assume $a+b=n$ and $a-b=1$.

In [4]:
def factor(n, primes):
    """Factor n using the Chinese Remainder Theorem.

    n: the number to factor
    primes: an array of primes
    Returns: a nontrivial factor of n, or None on failure
    """
    a_mod_m = (n-1) // 2
    b_mod_m = 1
    a = [[a_mod_m % p, -a_mod_m % p] for p in primes]
    b = [[b_mod_m % p, -b_mod_m % p] for p in primes]
    print(f"a = {a}")
    print(f"b = {b}")
    # get all possible a +/- b ("generalized", for however
    # many prime factors)
    a_mod_m = [solve_mod_primes(a1, primes)
        for a1 in itertools.product(*a)]
    b_mod_m = [solve_mod_primes(b1, primes)
        for b1 in itertools.product(*b)]
    # check GCD of each of these
    for (a1,b1) in itertools.product(a_mod_m, b_mod_m):
        f = math.gcd(n, a1+b1)
        if f != 1 and f != n:
            # print(f"f = {f}")
            return f
    return None


Some tests:

In [5]:
primes = [11,13,17]

In [6]:
factor(3*5, primes)

a = [[7, 4], [7, 6], [7, 10]]
b = [[1, 10], [1, 12], [1, 16]]


3

In [7]:
factor(5*7, primes)

a = [[6, 5], [4, 9], [0, 0]]
b = [[1, 10], [1, 12], [1, 16]]


5

In [8]:
primes = [11,13,17,19,23]

In [9]:
factor(29*31, primes)

a = [[9, 2], [7, 6], [7, 10], [12, 7], [12, 11]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22]]


29

In [10]:
primes = [11,13,17,19,23,29,31]

In [11]:
factor(37*41, primes)

a = [[10, 1], [4, 9], [10, 7], [17, 2], [22, 1], [4, 25], [14, 17]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]


37

In [12]:
factor(41*43, primes)

a = [[1, 10], [10, 3], [14, 3], [7, 12], [7, 16], [11, 18], [13, 18]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]


41

In [13]:
factor(3*47, primes)

a = [[4, 7], [5, 8], [2, 15], [13, 6], [1, 22], [12, 17], [8, 23]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]


3

In [14]:
factor(47*59, primes)

a = [[0, 0], [8, 5], [9, 8], [18, 1], [6, 17], [23, 6], [22, 9]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]


59

In [15]:
# just confirming that, even though 47*59 isn't divisible by 11,
# a == 1386 is
a = (47*59-1) // 2
a, a % 11

(1386, 0)

In [16]:
factor(61*67, primes)

a = [[8, 3], [2, 11], [3, 14], [10, 9], [19, 4], [13, 16], [28, 3]]
b = [[1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30]]


67

## Slightly more testing

It seems to work for small numbers. What about slightly larger numbers?

In [17]:
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def get_primes(n_primes):
    primes = []
    num = 2
    while len(primes) < n_primes:
        if is_prime(num):
            primes.append(num)
        num += 1
    return primes

primes = get_primes(1000)
display(primes[:10]) # display the first 10 primes

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

In [18]:
# use some small primes (starting with 7) for m
small_primes = primes[3:12]
print(small_primes)
larger_primes = primes[300:310]
m = math.prod(small_primes)
print(f"m = {m}")

for (a, b) in itertools.combinations(larger_primes, 2):
    n = a*b
    if n > m:
        continue
    print(f"n = {n} = {a} * {b}", flush=True)
    f = factor(n, small_primes)
    if f is not None:
        print(f"f = {f}\n", flush=True)
    else:
        print(f"failed to factor {n} = {a}*{b}", flush=True)
        break

[7, 11, 13, 17, 19, 23, 29, 31, 37]
m = 247357937827
n = 3980021 = 1993 * 1997
a = [[1, 6], [0, 0], [9, 4], [7, 10], [7, 12], [4, 19], [1, 28], [27, 4], [2, 35]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30], [1, 36]]
f = 1997

n = 3984007 = 1993 * 1999
a = [[6, 1], [2, 9], [0, 0], [11, 6], [5, 14], [19, 4], [22, 7], [5, 26], [34, 3]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30], [1, 36]]
f = 1993

n = 3991979 = 1993 * 2003
a = [[2, 5], [6, 5], [8, 5], [2, 15], [1, 18], [3, 20], [6, 23], [23, 8], [24, 13]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30], [1, 36]]
f = 2003

n = 4007923 = 1993 * 2011
a = [[1, 6], [3, 8], [11, 2], [1, 16], [12, 7], [17, 6], [3, 26], [28, 3], [4, 33]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22], [1, 28], [1, 30], [1, 36]]
f = 2011

n = 4019881 = 1993 * 2017
a = [[2, 5], [9, 2], [10, 3], [13, 4], [6, 13], [16, 7], [8, 21], [24, 7], [26, 11]]
b = [[1, 6], [1, 1

However, if we don't use "enough" prime factors, using $a-b=1$ sometimes doesn't work:

In [19]:
# use some small primes (starting with 7) for m
small_primes = primes[3:9]
print(small_primes)
larger_primes = primes[302:305]
m = math.prod(small_primes)
print(f"m = {m}")

for (a, b) in itertools.combinations(larger_primes, 2):
    n = a*b
    if n > m:
        continue
    print(f"n = {n} = {a} * {b}", flush=True)
    f = factor(n, small_primes)
    if f is not None:
        print(f"f = {f}\n", flush=True)
    else:
        print(f"failed to factor {n} = {a}*{b}", flush=True)
        break

[7, 11, 13, 17, 19, 23]
m = 7436429
n = 4003997 = 1999 * 2003
a = [[5, 2], [9, 2], [11, 2], [10, 7], [6, 13], [9, 14]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22]]
f = 1999

n = 4019989 = 1999 * 2011
a = [[0, 0], [8, 3], [12, 1], [16, 1], [3, 16], [1, 22]]
b = [[1, 6], [1, 10], [1, 12], [1, 16], [1, 18], [1, 22]]
failed to factor 4019989 = 1999*2011


We could guarantee that this would always work, by trying many random values of $a+b$ and $a-b$ -- eventually, we'd just find those which are factors of $n$! This obviously might take a while...

# Questions

- With $a$ and $b$ chosen with $a-b=1$, how often will at least one of the "generalized $a+b$" numbers have a nontrivial GCD with $n$? (It works for some small examples, but not larger examples.)

- What value of $m = \prod p_i$ works best? Presumably there are some trade-offs here.

- Given that the number of "generalized $a+b$" numbers grows like $2^{|P|}$, is this practical? (Presumably not.)

- What is this most similar to? (Gemini suggests the quadratic sieve, which seems plausible.)
